In [1]:
import matplotlib.pyplot as plt

In [2]:
#====================
# Load Utils ========
#====================

import numpy as np
import uproot as ur
import awkward as ak
import time as t
import os
print("Awkward version: "+str(ak.__version__))
print("Uproot version: "+str(ur.__version__))

Awkward version: 1.4.0
Uproot version: 4.0.11


In [3]:
#====================
# Metadata ==========
#====================
track_branches = ['trackEta_EMB1', 'trackPhi_EMB1', 'trackEta_EMB2', 'trackPhi_EMB2', 'trackEta_EMB3', 'trackPhi_EMB3',
                  'trackEta_TileBar0', 'trackPhi_TileBar0', 'trackEta_TileBar1', 'trackPhi_TileBar1',
                  'trackEta_TileBar2', 'trackPhi_TileBar2']

event_branches = ["cluster_nCells", "cluster_cell_ID", "cluster_cell_E", 'cluster_nCells', "nCluster", "eventNumber",
                  "nTrack", "nTruthPart", "truthPartPdgId", "cluster_Eta", "cluster_Phi", 'trackPt', 'trackP',
                  'trackMass', 'trackEta', 'trackPhi', 'truthPartE', 'cluster_ENG_CALIB_TOT', "cluster_E", 'truthPartPt']

ak_event_branches = ["cluster_nCells", "cluster_cell_ID", "cluster_cell_E", "cluster_nCells",
                  "nTruthPart", "truthPartPdgId", "cluster_Eta", "cluster_Phi", "trackPt", "trackP",
                  "trackMass", "trackEta", "trackPhi", "truthPartE", "cluster_ENG_CALIB_TOT", "cluster_E", "truthPartPt"]

np_event_branches = ["nCluster", "eventNumber", "nTrack", "nTruthPart"]

geo_branches = ["cell_geo_ID", "cell_geo_eta", "cell_geo_phi", "cell_geo_rPerp", "cell_geo_sampling"]


# The functions

In [4]:
def dict_from_tree(tree, branches=None, np_branches=None):
    ''' Loads branches as default awkward arrays and np_branches as numpy arrays. '''
    dictionary = dict()
    if branches is not None:
        for key in branches:
            branch = tree.arrays()[key]
            dictionary[key] = branch
            
    if np_branches is not None:
        for np_key in np_branches:
            np_branch = np.ndarray.flatten(tree.arrays()[np_key].to_numpy())
            dictionary[np_key] = np_branch
    
    if branches is None and np_branches is None:
        raise ValueError("No branches passed to function.")
        
    return dictionary

In [5]:
def find_max_dim_tuple(events, event_dict):
    nEvents = len(events)
    max_clust = 0
    
    for i in range(nEvents):
        event = events[i,0]
        track_nums = events[i,1]
        clust_nums = events[i,2]
        
        clust_num_total = 0
        # set this to six for now to handle single track events, change later
        track_num_total = 6
        
        # Check if there are clusters, None type object may be associated with it
        if clust_nums is not None:
            # Search through cluster indices
            for clst_idx in clust_nums:
                nInClust = len(event_dict['cluster_cell_ID'][event][clst_idx])
                # add the number in each cluster to the total
                clust_num_total += nInClust

        total_size = clust_num_total + track_num_total
        if total_size > max_clust:
            max_clust = total_size
    
    # 6 for energy, eta, phi, rperp, track flag, sample layer
    return (nEvents, max_clust, 6)

In [6]:
def find_index_1D(values, dictionary):
    ''' Use a for loop and a dictionary. values are the IDs to search for. dict must be in format 
    (cell IDs: index) '''
    idx_vec = np.zeros(len(values), dtype=np.int32)
    for i in range(len(values)):
        idx_vec[i] = dictionary[values[i]]
    return idx_vec

# The data files

In [7]:
#====================
# File setup ========
#====================
# user.angerami.24559744.OutputStream._000001.root
# Number of files
Nfile = 1
fileNames = []
file_prefix = 'user.angerami.24559744.OutputStream._000'
for i in range(1,Nfile+1):
    endstring = f'{i:03}'
    fileNames.append(file_prefix + endstring + '.root')


In [8]:
#====================
# Load Data Files ===
#====================

## GEOMETRY DICTIONARY ##
geo_file = ur.open('/fast_scratch/atlas_images/v01-45/cell_geo.root')
CellGeo_tree = geo_file["CellGeo"]
geo_dict = dict_from_tree(tree=CellGeo_tree, branches=None, np_branches=geo_branches)

# cell geometry data
cell_geo_ID = geo_dict['cell_geo_ID']
cell_ID_dict = dict(zip(cell_geo_ID, np.arange(len(cell_geo_ID))))

# for event dictionary
events_prefix = '/fast_scratch/atlas_images/v01-45/pipm/'

# Use this to compare with the dimensionality of new events
firstArray = True

# Loop over files: 

In [9]:
k = 1 # tally used to keep track of file number
tot_nEvts = 0 # used for keeping track of total number of events
max_nPoints = 0 # used for keeping track of the largest 'point cloud'
t_tot = 0 # total time

In [10]:
#for currFile in fileNames:
#    
#    # Check for file, a few are missing
#    if not os.path.isfile(events_prefix+currFile):
#        print()
#        print('File '+events_prefix+currFile+' not found..')
#        print()
##        k += 1
#        continue
#    
#    else:
#        print()
#        print('Working on File: '+str(currFile)+' - '+str(k)+'/'+str(Nfile))
#        k += 1

Just test one file

In [11]:
currFile = 'user.angerami.24559744.OutputStream._000001.root'

In [12]:
## EVENT DICTIONARY ##
event = ur.open(events_prefix+currFile)
event_tree = event["EventTree"]
event_dict = dict_from_tree(tree=event_tree, branches=ak_event_branches, np_branches=np_event_branches)

ak_event_branches -> Cluster, tracks, etcs. arrays per event

np_branches -> Event-level variables. one value per event

In [13]:
ak_event_branches

['cluster_nCells',
 'cluster_cell_ID',
 'cluster_cell_E',
 'cluster_nCells',
 'nTruthPart',
 'truthPartPdgId',
 'cluster_Eta',
 'cluster_Phi',
 'trackPt',
 'trackP',
 'trackMass',
 'trackEta',
 'trackPhi',
 'truthPartE',
 'cluster_ENG_CALIB_TOT',
 'cluster_E',
 'truthPartPt']

In [14]:
np_event_branches

['nCluster', 'eventNumber', 'nTrack', 'nTruthPart']

In [15]:
import collections
elements_count = collections.Counter(event_dict['nCluster'])
for key, value in elements_count.items():
    if key == 0:
       print(f"Events with {key} clusters: {value}")
print("Total number of events: ", len(event_dict['nCluster']))

Events with 0 clusters: 6597
Total number of events:  20000


# Event level cuts

In [16]:
#===================
# APPLY CUTS =======
#===================
# create ordered list of events to use for index slicing
nEvents = len(event_dict['eventNumber'])
all_events = np.arange(0,nEvents,1,dtype=np.int32) #array with event index

In [17]:
nCluster = event_dict['nCluster']

In [18]:
filtered_event_mask = nCluster != 0

In [19]:
filtered_event = all_events[filtered_event_mask]

In [20]:
print("* selected events: ", len(filtered_event), "/", len(all_events))

* selected events:  13403 / 20000


First event does not have clusters and was filtered out:

In [21]:
filtered_event

array([    1,     3,     4, ..., 19997, 19998, 19999], dtype=int32)

In [22]:
nCluster[0]

0

Second event have one cluster with 105 cells

In [23]:
ncells=event_dict['cluster_nCells']
ncells[8]

<Array [161, 64] type='2 * int32'>

# Loop over events:

In [24]:
#============================================#
## CREATE INDEX ARRAY FOR  CLUSTERS ##
#============================================#
event_indices = []
t0 = t.time()

In [25]:
#for evt in filtered_event:

In [26]:
evt =5000   #Just one event for testing

In [27]:
# pull cluster number, don't need zero index as it's loaded as a np array
nClust = event_dict["nCluster"][evt]
cluster_idx = np.arange(nClust)

In [28]:
## ENERGY SELECTION AND ETA SELECTION
clusterEs = event_dict["cluster_E"][evt].to_numpy()

In [29]:
clus_phi = event_dict["cluster_Phi"][evt].to_numpy()
clus_eta = event_dict["cluster_Eta"][evt].to_numpy()

In [30]:
eta_mask = abs(clus_eta) < 0.7
e_mask = clusterEs > 0.5

selection = eta_mask & e_mask

np.argmax returns the first instance of True in an array. But that is useful just for the one cluster case

In [31]:
selection

array([ True, False,  True,  True])

In [32]:
selected_clusters = cluster_idx[selection]

In [33]:
selected_clusters

array([0, 2, 3])

In [34]:
## CREATE LIST ##
# Note: currently do not have track only events. Do this in the future    
if np.count_nonzero(selection) > 0:
    event_indices.append((evt, 0, selected_clusters))
    print("* Event ", evt, " has ", np.count_nonzero(selection), " /" ,nClust,"selected clusters")

* Event  5000  has  3  / 4 selected clusters


In [35]:
event_indices

[(5000, 0, array([0, 2, 3]))]

## End the Loop over events

In [36]:
event_indices = np.array(event_indices, dtype=np.object_)

In [37]:
event_indices

array([[5000, 0, array([0, 2, 3])]], dtype=object)

# Check the dimension of the output X per file:

In [38]:
#=========================#
## DIMENSIONS OF X ARRAY ##
#=========================#
t0 = t.time()
max_dims = find_max_dim_tuple(event_indices, event_dict)

Returns (total number of selected events, total number of cells, 6 features)

Features: 6 for energy, eta, phi, rperp, track flag, sample layer

In [39]:
evt_tot = max_dims[0]
tot_nEvts += max_dims[0]

In [40]:
evt_tot

1

In [41]:
tot_nEvts

1

In [42]:
    # keep track of the largest point cloud to use for saving later
    if max_dims[1] > max_nPoints:
        max_nPoints = max_dims[1]

In [43]:
max_dims[1]

460

In [44]:
max_nPoints

460

In [45]:
    print('Selected number of events: '+str(evt_tot))
    print('Number of cells: '+str(max_dims[1]))

Selected number of events: 1
Number of cells: 460


In [46]:
    # Create arrays
    Y_new = np.zeros((max_dims[0],3))
    X_new = np.zeros(max_dims)

In [47]:
    print('* Events with selected clusters: '+str(max_dims[0]))
    print('* Total number of cells: '+str(max_dims[1]))
    print('* Dim of largest point cloud: '+str(max_nPoints))

* Events with selected clusters: 1
* Total number of cells: 460
* Dim of largest point cloud: 460


# Fill the arrays

Loop over selected-selected events


Recall: event_indices.append((evt, 0, selected_clusters))

In [77]:
    #===================#
    ## FILL IN ENTRIES ##==============================================================
    #===================#
    t0 = t.time()
    for i in range(max_dims[0]):
        # pull all relevant indices
        evt = event_indices[i,0]
        track_idx = event_indices[i,1]
        # recall this now returns an array
        cluster_nums = event_indices[i,2]
        
        
        print("evt",evt)
        print("track_idx",track_idx)
        print("cluster_nums",cluster_nums)
        
        ##############
        ## CLUSTERS ##
        ##############
        # set up to have no clusters, further this with setting up the same thing for tracks
        target_ENG_CALIB_TOT = -1
        if cluster_nums is not None:
            print("there is a cluster here")
            cluster_Eta = event_dict['cluster_Eta'][evt].to_numpy()
            cluster_Phi = event_dict['cluster_Phi'][evt].to_numpy()
            av_Eta = np.mean(cluster_Eta)
            av_Phi = np.mean(cluster_Phi)

            nClust_current_total = 0
            target_ENG_CALIB_TOT = 0
            
            print(" Average eta in the event", av_Eta)
            print(" Average phi in the event", av_Phi)
            
            for c in cluster_nums:
                # cluster data
                target_ENG_CALIB_TOT += event_dict['cluster_ENG_CALIB_TOT'][evt][c]
                cluster_cell_ID = event_dict['cluster_cell_ID'][evt][c].to_numpy()
                nInClust = len(cluster_cell_ID)
                cluster_cell_E = event_dict['cluster_cell_E'][evt][c].to_numpy()            
                cell_indices = find_index_1D(cluster_cell_ID, cell_ID_dict)
                
                print("----Cluster ",c )
                print("- Number of cells: ",nInClust)
                print("- eta: ",event_dict['cluster_Eta'][evt][c])
                print("- phi: ",event_dict['cluster_Phi'][evt][c])
                               
                cluster_cell_Eta = geo_dict['cell_geo_eta'][cell_indices]
                cluster_cell_Phi = geo_dict['cell_geo_phi'][cell_indices]
                cluster_cell_rPerp = geo_dict['cell_geo_rPerp'][cell_indices]
                cluster_cell_sampling = geo_dict['cell_geo_sampling'][cell_indices]
                
                print("---- - First cell in cluster ",c )
                print("- - eta: ",cluster_cell_Eta[0])
                print("- - phi: ",cluster_cell_Phi[0])
                print("- - eta normalised to average: ",cluster_cell_Eta[0]- av_Eta)
                print("- - phi normalised to average: ",cluster_cell_Phi[0]- av_Phi)
                print("- - eta normalised to cluster:",cluster_cell_Eta[0]-event_dict['cluster_Eta'][evt][c])
                print("- - phi normalised to cluster: ",cluster_cell_Phi[0]-event_dict['cluster_Phi'][evt][c])
                
                # input all the data
                # note here we leave the fourth entry zeros (zero for flag!!!)
                low = nClust_current_total
                high = low + nInClust
                X_new[i,low:high,0] = np.log(cluster_cell_E)
                # Normalize to average cluster centers
                X_new[i,low:high,1] = cluster_cell_Eta - av_Eta #cluster_cell_Eta - event_dict['cluster_Eta'][evt][c]
                X_new[i,low:high,2] = cluster_cell_Phi - av_Phi #cluster_cell_Phi -event_dict['cluster_Phi'][evt][c]
                X_new[i,low:high,3] = cluster_cell_rPerp
                X_new[i,low:high,5] = cluster_cell_sampling * 0.1
                
                nClust_current_total += nInClust

evt 5000
track_idx 0
cluster_nums [0 2 3]
there is a cluster here
 Average eta in the event -0.62879086
 Average phi in the event 0.35715285
----Cluster  0
- Number of cells:  438
- eta:  -0.608532726764679
- phi:  0.2467040866613388
---- - First cell in cluster  0
- - eta:  -0.6113232
- - phi:  0.23482959
- - eta normalised to average:  0.017467678
- - phi normalised to average:  -0.12232326
- - eta normalised to cluster: -0.0027904510498046875
- - phi normalised to cluster:  -0.011874496936798096
----Cluster  2
- Number of cells:  13
- eta:  -0.6112126111984253
- phi:  0.4098142981529236
---- - First cell in cluster  2
- - eta:  -0.6114513
- - phi:  0.40671355
- - eta normalised to average:  0.017339528
- - phi normalised to average:  0.049560696
- - eta normalised to cluster: -0.00023871660232543945
- - phi normalised to cluster:  -0.003100752830505371
----Cluster  3
- Number of cells:  3
- eta:  -0.511576771736145
- phi:  0.3450736105442047
---- - First cell in cluster  3
- - eta: 

In [76]:
nClust_current_total

454

In [78]:
event_dict['cluster_cell_E'][evt][c].to_numpy() 

array([0.71349967, 0.00902699, 0.02797122])